# Dataset AdventureWorksDW

### Operações SQL simples

A base **AdventureWorksDW** https://github.com/microsoft/sql-server-samples/ é bastante conhecida no mundo de dados Microsoft.

Vamos fazer algumas atividades com o SparkSQL, como desafios.

### Carregando o PySpark

In [ ]:
# !pip install pyspark

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [ ]:
conf = SparkConf().setMaster('local').setAppName('PySpark SQL')
sc = SparkContext.getOrCreate(conf = conf)

Criação do contexto do objeto SparkSQL que será responsável por executar as *query* do Spark com comandos SQL.

In [ ]:
sql = SQLContext(sc)

In [ ]:
sql

Aqui é a criação de um Dataframe com os dados que estamos lendo do arquivo CSV. Usamos o contexto do SparkSQL, mas ainda sim é um Dataframe

In [ ]:
FactInternetSales_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/FactInternetSales.csv')
DimSalesTerritory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimSalesTerritory.csv')
DimProductSubcategory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProductSubcategory.csv')
DimProductCategory_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProductCategory.csv')
DimProduct_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimProduct.csv')
DimCustomer_Spark = sql.read.format("csv").options(header='true').load('AdventureWorksDW/DimCustomer.csv')

In [ ]:
DimSalesTerritory_Spark.printSchema()

In [ ]:
FactInternetSales_Spark.select(["ProductKey","TotalProductCost"]).show(10)

A partir do dataframe **dadosSpark**, vamos registrar uma tabela temporária do SQL, chamada **Carros**

In [ ]:
FactInternetSales_Spark.registerTempTable("FactInternetSales")
DimSalesTerritory_Spark.registerTempTable("DimSalesTerritory")
DimProductSubcategory_Spark.registerTempTable("DimProductSubcategory")
DimProductCategory_Spark.registerTempTable("DimProductCategory")
DimProduct_Spark.registerTempTable("DimProduct")
DimCustomer_Spark.registerTempTable("DimCustomer")

A partir do motor do **SparkSQL** vamos escrever uma *query* em SQL que retornará todas as linhas e colunas da nossa base

In [ ]:
sql.sql("SELECT sub.*, cat.EnglishProductCategoryName FROM \
            DimProductSubcategory as SUB inner join \
            DimProductCategory Cat \
                on SUB.ProductCategoryKey = Cat.ProductCategoryKey \
            where Cat.ProductCategoryKey = 2").show()

In [ ]:
sql.sql("select * from DimProductCategory").show()

In [ ]:
sql.sql("select CatP.EnglishProductCategoryName as Categoria, SalT.SalesTerritoryCountry as Pais, round(sum(FIS.SalesAmount),2) AS TotalVendas from \
    FactInternetSales FIS inner join DimProduct AS Prod on FIS.ProductKey = Prod.ProductKey \
    inner join DimProductSubcategory AS SubC on Prod.ProductSubcategoryKey = SubC.ProductSubcategoryKey \
    inner join DimProductCategory AS CatP on SubC.ProductCategoryKey = CatP.ProductCategoryKey \
    inner join DimSalesTerritory AS SalT on FIS.SalesTerritoryKey = SalT.SalesTerritoryKey \
    group by CatP.EnglishProductCategoryName, SalT.SalesTerritoryCountry \
    order by SalT.SalesTerritoryCountry, CatP.EnglishProductCategoryName").show(10)

In [ ]:
sql.sql("").show()